In [47]:
import os
import pandas as pd
import datetime
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.simplefilter('ignore')

# 環境設定

In [3]:
DATA_PATH = "../data"

# 分析

## データ読み込み

In [37]:
train_data = pd.read_csv(os.path.join(DATA_PATH, "train.csv"), index_col=0)
test_data = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), index_col=0)

## 前処理

In [38]:
dollar_amount_cols = ["DisbursementGross", "GrAppv", "SBA_Appv"]
for col in dollar_amount_cols:
    train_data[col] = train_data[col].apply(lambda x: x.replace("$", "").replace(".", "").replace(",", "")).astype(int).copy()
    test_data[col] = test_data[col].apply(lambda x: x.replace("$", "").replace(".", "").replace(",", "")).astype(int).copy()

## 各カラムの確認

### nullのあるカラム

In [22]:
train_data.isnull().sum(axis=0)

Term                    0
NoEmp                   0
NewExist                0
CreateJob               0
RetainedJob             0
FranchiseCode           0
RevLineCr            1079
LowDoc                531
DisbursementDate      150
MIS_Status              0
Sector                  0
ApprovalDate            0
ApprovalFY              0
City                    0
State                   0
BankState              11
DisbursementGross       0
GrAppv                  0
SBA_Appv                0
UrbanRural              0
dtype: int64

### 表記揺れカラム

In [27]:
# RevLineCr: リボルビング信用枠か
print('train')
display(train_data["RevLineCr"].value_counts())
print('test')
display(test_data["RevLineCr"].value_counts())

train


N    27618
Y     7353
0     5561
T      696
Name: RevLineCr, dtype: int64

test


N    27829
Y     7454
0     5346
T      638
Name: RevLineCr, dtype: int64

In [28]:
# LowDoc: 15万ドル未満のローンの1ページの短い申請で処理できるプログラムか
print('train')
display(train_data["LowDoc"].value_counts())
print('test')
display(test_data["LowDoc"].value_counts())

train


N    34313
Y     5277
0      684
A      570
S      540
C      392
Name: LowDoc, dtype: int64

test


N    34360
Y     5275
0      675
A      561
S      523
C      394
Name: LowDoc, dtype: int64

In [33]:
print('train')
display(train_data[pd.to_datetime(train_data["ApprovalDate"]) >= pd.to_datetime("2024-03-01")])
print('test')
display(test_data[pd.to_datetime(test_data["ApprovalDate"]) >= pd.to_datetime("2024-03-01")])

train


,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
1417,238,22,1.0,0,0,0,N,N,26-Mar-10,1,62,17-Oct-73,1974,MILPITAS,CA,DC,44550000,44550000,33412500,0
8682,84,17,1.0,0,0,0,N,N,5-Oct-88,1,44,17-Oct-73,1974,SONORA,CA,CA,18000000,18000000,18000000,0
18156,83,49,1.0,0,0,0,N,N,12-Feb-88,1,62,17-Oct-73,1974,JACKSON HEIGHTS,NY,SD,6700000,5000000,2500000,0
28238,240,48,1.0,9,0,0,N,N,31-Dec-07,1,72,17-Oct-73,1974,DASSEL,MN,SD,37820000,37820000,28365000,0


test


,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
49650,303,24,1.0,0,0,0,N,N,28-Feb-05,62,17-Oct-73,1974,CLAREMORE,OK,OK,94050000,94050000,70537500,0
71059,83,21,1.0,0,0,0,N,N,31-Jan-01,42,17-Oct-73,1974,BOCA RATON,FL,FL,7834800,5000000,2500000,0
78536,239,42,1.0,1,0,0,N,N,28-Feb-05,44,17-Oct-73,1974,FAIRHAVEN,MA,RI,6718400,5000000,2500000,0


In [34]:
print('train')
display(train_data[pd.to_datetime(train_data["ApprovalDate"], format="%d-%b-%y") >= pd.to_datetime("2024-03-01")])
print('test')
display(test_data[pd.to_datetime(test_data["ApprovalDate"], format="%d-%b-%y") >= pd.to_datetime("2024-03-01")])

train


,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural


test


,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural


### 年月等

In [39]:
# 前処理
train_data["ApprovalDate"] = pd.to_datetime(train_data["ApprovalDate"], format="%d-%b-%y")
train_data["DisbursementDate"] = pd.to_datetime(train_data["DisbursementDate"], format="%d-%b-%y")
test_data["ApprovalDate"] = pd.to_datetime(test_data["ApprovalDate"], format="%d-%b-%y")
test_data["DisbursementDate"] = pd.to_datetime(test_data["DisbursementDate"], format="%d-%b-%y")

In [40]:
train_data["WatingTime"] = train_data["DisbursementDate"] - train_data["ApprovalDate"]
test_data["WatingTime"] = test_data["DisbursementDate"] - test_data["ApprovalDate"]

In [49]:
cols=["ApprovalDate", "DisbursementDate"]
display(train_data[cols].describe(include="all"))
display(test_data[cols].describe(include="all"))

,ApprovalDate,DisbursementDate
count,42307,42157
unique,3868,916
top,2003-10-07 00:00:00,2003-11-30 00:00:00
freq,1347,1634
first,1973-10-17 00:00:00,1973-12-06 00:00:00
last,2014-01-13 00:00:00,2014-07-05 00:00:00


,ApprovalDate,DisbursementDate
count,42308,42148
unique,3888,915
top,2003-10-07 00:00:00,2003-11-30 00:00:00
freq,1372,1627
first,1973-10-17 00:00:00,1973-12-06 00:00:00
last,2014-01-13 00:00:00,2014-07-05 00:00:00


In [50]:
cols = ["WatingTime"]
display(train_data[cols].describe(include="all"))
display(test_data[cols].describe(include="all"))

,WatingTime
count,42157
mean,-72 days +01:21:50.558151671
std,3103 days 20:36:13.473957312
min,-12152 days +00:00:00
25%,-2167 days +00:00:00
50%,-123 days +00:00:00
75%,2046 days 00:00:00
max,13309 days 00:00:00


,WatingTime
count,42148
mean,-84 days +19:06:31.192939167
std,3110 days 12:51:07.796482592
min,-12452 days +00:00:00
25%,-2151 days +18:00:00
50%,-118 days +00:00:00
75%,2062 days 00:00:00
max,12250 days 00:00:00
